# 第5章 LangChain多任务应用开发

## 💡 学习目标

1. 如何使用 LangChain：一套在大模型能力上封装的工具框架
2. 如何用几行代码实现一个复杂的 AI 应用
3. 面向大模型的流程开发的过程抽象

## 写在前面

- LangChain 也是一套面向大模型的开发框架（SDK）
- LangChain 是 AGI 时代软件工程的一个探索和原型
- 学习 LangChain 要关注接口变更


## LangChain 的核心组件

1. 模型 I/O 封装
   - Chat Models：对语言模型接口的封装
   - PromptTemple：提示词模板
   - OutputParser：解析输出
2. 数据连接封装（弱于 LlamaIndex）
   - Document Loaders：各种格式文件的加载器
   - Document Transformers：对文档的常用操作，如：split, filter, translate, extract metadata, etc
   - Text Embedding Models：文本向量化表示，用于检索等操作
   - Verctorstores & Retrievers：向量数据库与向量检索
3. 架构封装
   - Chain/LCEL：实现一个功能或者一系列顺序功能组合
   - Agent：根据用户输入，自动规划执行步骤，自动选择每步需要的工具，最终完成用户指定的功能
     - Tools：调用外部功能的函数，例如：调 google 搜索、文件 I/O、Linux Shell 等等
   - LangGraph：工作流开发框架
5. LangSmith：过程监控与调试框架

<img src="./assets/langchain.svg" width="800px">


### 文档（以 Python 版为例）

- 功能模块：https://python.langchain.com/docs/tutorials
- API 文档：https://python.langchain.com/api_reference/
- 三方组件集成：https://python.langchain.com/docs/integrations/providers/
- 更多 HowTo：https://python.langchain.com/docs/how_to/


### LangChain 是开源项目

项目地址：https://github.com/langchain-ai

## 1. 模型 I/O 封装

把不同的模型，统一封装成一个接口，方便更换模型而不用重构代码。

### 1.1 模型 API: ChatModel


#### 1.1.1 OpenAI 模型封装

In [ ]:
# !pip install -U langchain
# !pip install -U langchain-openai

In [10]:
from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4o-mini", model_provider="openai")
response = model.invoke("你是谁")
print(response.content)

我是一个人工智能助手，旨在提供信息和回答问题。如果你有任何问题或需要帮助，随时可以问我！


#### 1.1.2 多轮对话 Session 封装

In [11]:
from langchain.schema import (
    AIMessage,  # 等价于OpenAI接口中的assistant role
    HumanMessage,  # 等价于OpenAI接口中的user role
    SystemMessage  # 等价于OpenAI接口中的system role
)

messages = [
    SystemMessage(content="你是聚客AI大模型课的课程助理。"),
    HumanMessage(content="我是聚客AI学员，我叫小聚。"),
    AIMessage(content="欢迎！"),
    HumanMessage(content="我是谁？")
]

ret = model.invoke(messages)

print(ret.content)

你是聚客AI的学员，小聚。很高兴见到你！有什么我可以帮助你的吗？


<div class="alert alert-success">
<b>划重点：</b>通过模型封装，实现不同模型的统一接口调用
</div>


#### 1.1.3 DeepSeek 模型封装

In [16]:
# !pip install -U langchain-deepseek

In [1]:
from langchain.chat_models import init_chat_model

model = init_chat_model(model="deepseek-chat", model_provider="deepseek")

response = model.invoke("你是谁")
print(response.content)

你好！我是DeepSeek，由深度求索公司创造的AI助手！😊

我是一个纯文本模型，虽然不支持多模态识别功能，但我有文件上传功能，可以帮你处理图像、txt、pdf、ppt、word、excel等文件，并从中读取文字信息进行分析处理。我完全免费使用，拥有128K的上下文长度，还支持联网搜索功能（需要你在Web/App中手动点开联网搜索按键）。

你可以通过官方应用商店下载我的App来使用。我很乐意帮助你解答问题、处理文档、进行对话交流等等！

有什么我可以帮你的吗？无论是学习、工作还是日常问题，我都很愿意协助你！✨


#### 1.1.4 流式输出

In [2]:
for token in model.stream("你是谁"):
    print(token.content, end="")

你好！我是DeepSeek，由深度求索公司创造的AI助手！😊

我是一个纯文本模型，虽然不支持多模态识别功能，但我有文件上传功能，可以帮你处理图像、txt、pdf、ppt、word、excel等文件，并从中读取文字信息进行分析处理。

我的特点包括：
- 完全免费使用，没有收费计划
- 支持128K上下文长度
- 可以联网搜索（需要你手动点开联网搜索按键）
- 有官方App可以下载
- 知识截止到2024年7月

我很乐意帮助你解答问题、协助处理文档、进行对话交流等等！有什么我可以帮你的吗？无论是学习、工作还是日常问题，我都很愿意为你提供帮助！✨

#### 1.1.5 通义千问模型封装

1. OpenAI兼容接口

In [3]:
from langchain_openai import ChatOpenAI
import os

chatLLM = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus",  # 此处以qwen-plus为例，您可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    # other params...
)
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "你是谁？"}]
response = chatLLM.invoke(messages)
response

AIMessage(content='你好！我是通义千问（Qwen），阿里巴巴集团旗下的超大规模语言模型。我能够回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。如果你有任何问题或需要帮助，欢迎随时告诉我！😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 22, 'total_tokens': 88, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-da4d4dae-c604-99f5-9a0f-3dc739743d95', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bfe86-4e16-7363-ba8e-9222b8cabad3-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 22, 'output_tokens': 66, 'total_tokens': 88, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

2. DashScope 接口

In [ ]:
# !pip install langchain-community
# !pip install dashscope

In [28]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.messages import HumanMessage

chatLLM = ChatTongyi(
    model="qwen-max",   # 此处以qwen-max为例，您可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    streaming=True,
    # other params...
)
res = chatLLM.stream([HumanMessage(content="hi")], streaming=True)
for r in res:
    print("chat resp:", r.content)

chat resp: Hello
chat resp: ! How can
chat resp:  I assist you
chat resp:  today?
chat resp: 
chat resp: 


#### 1.1.6 Ollama模型封装

In [ ]:
# !pip install -U langchain-ollama

In [9]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="qwen3:8b",
    temperature=0.7,
    # other params...
)

messages = [
    (
        "system",
        "你是一名专业的翻译家，可以将用户的中文翻译为英文。",
    ),
    ("human", "我喜欢编程。"),
]
ai_msg = llm.invoke(messages)
print(ai_msg.content)

<think>
好的，用户让我翻译“我喜欢编程。”这句话。首先，我需要确认这句话的准确意思。用户可能是在表达对编程的兴趣或热情。接下来，我要考虑不同的翻译方式，确保自然且符合英语习惯。

首先，直译的话是“I like programming.” 这是最直接的翻译，但可能不够生动。用户可能希望表达更强烈的感情，比如“我热爱编程”或者“我对编程充满热情”。这时候可以考虑使用更丰富的词汇，比如“love”或者“am passionate about”。

另外，还要注意语法结构。中文的“喜欢”对应英文的“like”或“love”，但根据语境选择合适的词。如果用户只是简单表达兴趣，用“like”即可；如果想强调热情，用“love”更合适。

还要考虑用户的潜在需求。用户可能是在写简历、申请学校，或者只是日常交流。如果是正式场合，可能需要更专业的表达，比如“I enjoy programming”或者“I am passionate about programming.” 如果是日常对话，简单的“I like programming”就足够。

另外，用户可能有更深层次的需求，比如寻找编程学习资源，或者讨论编程相关的话题。这时候翻译可能需要更详细，但根据当前问题，用户只需要翻译这句话，所以不需要扩展。

还要检查是否有文化差异需要注意。中文中的“喜欢”在英文中可能有不同的程度，但通常“like”已经足够表达。如果用户想更强烈，可以建议使用“love”或“passionate about”。

最后，确保翻译准确且自然，符合英语母语者的表达习惯，避免直译导致的生硬。比如，中文的“我喜欢编程”在英文中更自然的表达是“I like programming”或者“I enjoy programming.”，而“love”可能显得过于强烈，除非用户特别强调。

总结一下，用户可能需要一个简洁且自然的翻译，所以给出几个选项，让用户选择最合适的。例如，直接翻译、更强烈的表达，或者更正式的说法。同时，确保没有语法错误，并且符合英语的使用习惯。
</think>

I like programming.  
（或更生动地表达：I'm passionate about programming.）


### 1.2 模型的输入与输出

<img src="./assets/model_io.jpg" style="margin-left: 0px" width=1400px>

#### 1.2.1 Prompt 模板封装

1. PromptTemplate 可以在模板中自定义变量


In [6]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template("给我讲个关于{subject}的笑话")
print("===Template===")
print(template)
print("===Prompt===")
print(template.format(subject='小明'))

===Template===
input_variables=['subject'] input_types={} partial_variables={} template='给我讲个关于{subject}的笑话'
===Prompt===
给我讲个关于小明的笑话


In [7]:
from langchain.chat_models import init_chat_model

# 定义 LLM
llm = init_chat_model("deepseek-chat", model_provider="deepseek")
# 通过 Prompt 调用 LLM
ret = llm.invoke(template.format(subject='小明'))
# 打印输出
print(ret.content)

小明上课时肚子疼，想拉肚子，但不好意思说，就写纸条传给同桌小红：“你能替我跟老师说我必须去厕所吗？我急！”

小红看了一眼，举手站起来大声说：“老师！小明说他憋不住了，再不去厕所就要拉在裤子里了！”

全班爆笑，老师忍着笑点头。小明满脸通红地往外跑，结果在门口绊了一跤，爬起来喊：“等等！我还没输！我还能坚持到厕所！”

后来全班给他起了个外号：“厕所马拉松冠军”。


2. ChatPromptTemplate 用模板表示的对话上下文


In [9]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.chat_models import init_chat_model
llm = init_chat_model("deepseek-chat", model_provider="deepseek", temperature=0)

template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("你是{product}的客服助手。你的名字叫{name}"),
        HumanMessagePromptTemplate.from_template("{query}")
    ]
)

prompt = template.format_messages(
    product="聚客AI大模型课程",
    name="小聚",
    query="你是谁"
)

print(prompt)

ret = llm.invoke(prompt)

print(ret.content)

[SystemMessage(content='你是聚客AI大模型课程的客服助手。你的名字叫小聚', additional_kwargs={}, response_metadata={}), HumanMessage(content='你是谁', additional_kwargs={}, response_metadata={})]
你好！我是小聚，聚客AI大模型课程的客服助手。有什么可以帮你的吗？😊


3. MessagesPlaceholder 把多轮对话变成模板


In [10]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

human_prompt = "Translate your answer to {language}."
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

chat_prompt = ChatPromptTemplate.from_messages(
    # variable_name 是 message placeholder 在模板中的变量名
    # 用于在赋值时使用
    [MessagesPlaceholder("history"), human_message_template]
)

In [11]:
from langchain_core.messages import AIMessage, HumanMessage

human_message = HumanMessage(content="Who is Elon Musk?")
ai_message = AIMessage(
    content="Elon Musk is a billionaire entrepreneur, inventor, and industrial designer"
)

messages = chat_prompt.format_prompt(
    # 对 "history" 和 "language" 赋值
    history=[human_message, ai_message], language="中文"
)

print(messages.to_messages())

[HumanMessage(content='Who is Elon Musk?', additional_kwargs={}, response_metadata={}), AIMessage(content='Elon Musk is a billionaire entrepreneur, inventor, and industrial designer', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='Translate your answer to 中文.', additional_kwargs={}, response_metadata={})]


In [12]:
result = llm.invoke(messages)
print(result.content)

埃隆·马斯克（Elon Musk）是一位出生于南非的企业家、工程师、发明家和投资者。他是多家具有全球影响力的科技公司的创始人或联合创始人，以其雄心勃勃的愿景和对多个行业的颠覆性创新而闻名。

以下是他的主要身份和成就：

**1. 主要创办的公司与角色：**
*   **SpaceX（太空探索技术公司）**：创始人、CEO兼首席工程师。目标是降低太空运输成本，最终实现人类在火星上殖民，使人类成为“多行星物种”。
*   **特斯拉（Tesla, Inc.）**：CEO兼产品架构师。推动了电动汽车的革命，并大力发展太阳能和清洁能源存储解决方案。
*   **X（原Twitter）**：所有者兼首席技术官。于2022年收购了这家社交媒体平台。
*   **Neuralink（神经连接公司）**：联合创始人。致力于开发脑机接口技术，旨在将人脑与计算机连接起来。
*   **The Boring Company（钻孔公司）**：创始人。专注于建设地下隧道网络以解决城市交通拥堵问题。
*   **xAI**：创始人。于2023年成立的人工智能公司，旨在“理解宇宙的真实本质”。

**2. 早期成功：**
在涉足航天和汽车领域之前，他早期因共同创立了**Zip2**（一家为新闻机构提供城市指南软件的公司）和**PayPal**（全球领先的在线支付系统）而获得第一桶金和声誉。

**3. 个人特点与公众形象：**
*   **愿景宏大**：他的目标通常着眼于解决人类面临的重大挑战（如可持续能源、太空探索、人工智能安全）。
*   **高调活跃**：他非常频繁地在社交媒体（特别是X平台）上发声，发表对公司、技术、社会议题的看法，风格直接且常引发争议。
*   **工作狂**：以极度投入工作和设定激进的 deadlines 而著称。
*   **争议性**：他的管理风格、公开言论、对加密货币市场的影响以及收购Twitter后的改革举措等都常常成为媒体和公众讨论的焦点。

**总结来说，埃隆·马斯克是当今世界上最具影响力、同时也最具争议的科技领袖之一。他通过特斯拉加速了全球汽车行业向电动化的转型，通过SpaceX重塑了航天产业，并正通过他的其他公司尝试在交通、人机交互和人工智能等领域进行新的突破。**


<div class="alert alert-success">
<b>划重点：</b>把Prompt模板看作带有参数的函数
</div>


#### 1.2.2 从文件加载 Prompt 模板

In [11]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_file("example_prompt_template.txt", encoding="utf-8")
print("===Template===")
print(template)
print("===Prompt===")
print(template.format(topic='黑色幽默'))

===Template===
input_variables=['topic'] input_types={} partial_variables={} template='举一个关于{topic}的例子'
===Prompt===
举一个关于黑色幽默的例子


### 1.3 结构化输出

#### 1.3.1 直接输出 Pydantic 对象

In [12]:
from pydantic import BaseModel, Field

# 定义你的输出对象
class Date(BaseModel):
    year: int = Field(description="Year")
    month: int = Field(description="Month")
    day: int = Field(description="Day")
    era: str = Field(description="BC or AD")

In [13]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

from langchain.chat_models import init_chat_model
llm = init_chat_model("deepseek-chat", model_provider="deepseek")
# llm = init_chat_model("gpt-4o", model_provider="openai")
# 定义结构化输出的模型
structured_llm = llm.with_structured_output(Date)

template = """提取用户输入中的日期。
用户输入:
{query}"""

prompt = PromptTemplate(
    template=template,
)

query = "2023年四月6日天气晴..."
input_prompt = prompt.format_prompt(query=query)

structured_llm.invoke(input_prompt)

Date(year=2023, month=4, day=6, era='AD')

#### 1.3.2 输出指定格式的 JSON

In [14]:
# OpenAI 模型的JSON格式
json_schema = {
    "title": "Date",
    "description": "Formated date expression",
    "type": "object",
    "properties": {
        "year": {
            "type": "integer",
            "description": "year, YYYY",
        },
        "month": {
            "type": "integer",
            "description": "month, MM",
        },
        "day": {
            "type": "integer",
            "description": "day, DD",
        },
        "era": {
            "type": "string",
            "description": "BC or AD",
        },
    },
}
structured_llm = llm.with_structured_output(json_schema)

structured_llm.invoke(input_prompt)

{'year': 2023, 'month': 4, 'day': 6}

#### 1.3.3 使用 OutputParser

[`OutputParser`](https://python.langchain.com/v0.2/docs/concepts/#output-parsers) 可以按指定格式解析模型的输出

In [15]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser(pydantic_object=Date)

prompt = PromptTemplate(
    template="提取用户输入中的日期。\n用户输入:{query}\n{format_instructions}",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

input_prompt = prompt.format_prompt(query=query)
output = llm.invoke(input_prompt)
print("原始输出:\n"+output.content)

print("\n解析后:")
parser.invoke(output)

原始输出:
{"year": 2023, "month": 4, "day": 6, "era": "AD"}

解析后:


{'year': 2023, 'month': 4, 'day': 6, 'era': 'AD'}

也可以用 `PydanticOutputParser`


In [16]:
from langchain_core.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Date)

input_prompt = prompt.format_prompt(query=query)
output = llm.invoke(input_prompt)
print("原始输出:\n"+output.content)

print("\n解析后:")
parser.invoke(output)

原始输出:
{"year": 2023, "month": 4, "day": 6, "era": "AD"}

解析后:


Date(year=2023, month=4, day=6, era='AD')

`OutputFixingParser` 利用大模型做格式自动纠错

In [25]:
from langchain.output_parsers import OutputFixingParser
from langchain.chat_models import init_chat_model

# llm = init_chat_model(model="gpt-4o-mini", model_provider="openai")
llm = init_chat_model(model="deepseek-chat", model_provider="deepseek")

# 纠错能力与大模型能力相关
new_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

bad_output = output.content.replace("4","四")
print("PydanticOutputParser:")
try:
    parser.invoke(bad_output)
except Exception as e:
    print(e)

print("OutputFixingParser:")
new_parser.invoke(bad_output)

ModuleNotFoundError: No module named 'langchain.output_parsers'

### 1.4 Function Calling

In [26]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b

@tool
def multiply(a: float, b: float) -> float:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

In [29]:
import json

llm_with_tools = llm.bind_tools([add, multiply])

query = "3.5的4倍是多少?"
messages = [HumanMessage(query)]

output = llm_with_tools.invoke(messages)

print(json.dumps(output.tool_calls, indent=4))

[
    {
        "name": "multiply",
        "args": {
            "a": 3.5,
            "b": 4
        },
        "id": "call_00_asRxPcyH6nUzCzLiBdwumvl8",
        "type": "tool_call"
    }
]


回传 Funtion Call 的结果

In [30]:
messages.append(output)

available_tools = {"add": add, "multiply": multiply}

for tool_call in output.tool_calls:
    selected_tool = available_tools[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

new_output = llm_with_tools.invoke(messages)
for message in messages:
    print(json.dumps(message.model_dump(), indent=4, ensure_ascii=False))
print(new_output.content)

{
    "content": "3.5的4倍是多少?",
    "additional_kwargs": {},
    "response_metadata": {},
    "type": "human",
    "name": null,
    "id": null
}
{
    "content": "我来帮你计算3.5的4倍是多少。",
    "additional_kwargs": {
        "refusal": null
    },
    "response_metadata": {
        "token_usage": {
            "completion_tokens": 71,
            "prompt_tokens": 399,
            "total_tokens": 470,
            "completion_tokens_details": null,
            "prompt_tokens_details": {
                "audio_tokens": null,
                "cached_tokens": 192
            },
            "prompt_cache_hit_tokens": 192,
            "prompt_cache_miss_tokens": 207
        },
        "model_provider": "deepseek",
        "model_name": "deepseek-chat",
        "system_fingerprint": "fp_eaab8d114b_prod0820_fp8_kvcache",
        "id": "7e9230f8-4d33-49f8-8e5d-9b993e66caee",
        "finish_reason": "tool_calls",
        "logprobs": null
    },
    "type": "ai",
    "name": null,
    "id": "lc_run--019b

### 1.5 小结

1. LangChain 统一封装了各种模型的调用接口，包括补全型和对话型两种
2. LangChain 提供了 PromptTemplate 类，可以自定义带变量的模板
3. LangChain 提供了一些列输出解析器，用于将大模型的输出解析成结构化对象
4. LangChain 提供了 Function Calling 的封装
5. 上述模型属于 LangChain 中较为实用的部分

## 2. 数据连接封装

<img src="./assets/data_connection.jpg" style="margin-left: 0px" width=1400px>

### 2.1 文档加载器：Document Loaders


In [ ]:
# !pip install -U langchain-community pymupdf

In [31]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("./data/deepseek-v3-1-4.pdf")
pages = loader.load_and_split()

print(pages[0].page_content)

DeepSeek-V3 Technical Report
DeepSeek-AI
research@deepseek.com
Abstract
We present DeepSeek-V3, a strong Mixture-of-Experts (MoE) language model with 671B total
parameters with 37B activated for each token. To achieve efficient inference and cost-effective
training, DeepSeek-V3 adopts Multi-head Latent Attention (MLA) and DeepSeekMoE architec-
tures, which were thoroughly validated in DeepSeek-V2. Furthermore, DeepSeek-V3 pioneers
an auxiliary-loss-free strategy for load balancing and sets a multi-token prediction training
objective for stronger performance. We pre-train DeepSeek-V3 on 14.8 trillion diverse and
high-quality tokens, followed by Supervised Fine-Tuning and Reinforcement Learning stages to
fully harness its capabilities. Comprehensive evaluations reveal that DeepSeek-V3 outperforms
other open-source models and achieves performance comparable to leading closed-source
models. Despite its excellent performance, DeepSeek-V3 requires only 2.788M H800 GPU hours
for its full trai

### 2.2 文档处理器

#### 2.2.1 TextSplitter

In [ ]:
# !pip install --upgrade langchain-text-splitters

In [32]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=200, 
    length_function=len,
    add_start_index=True,
)

paragraphs = text_splitter.create_documents([pages[0].page_content])
for para in paragraphs:
    print(para.page_content)
    print('-------')

DeepSeek-V3 Technical Report
DeepSeek-AI
research@deepseek.com
Abstract
We present DeepSeek-V3, a strong Mixture-of-Experts (MoE) language model with 671B total
parameters with 37B activated for each token. To achieve efficient inference and cost-effective
training, DeepSeek-V3 adopts Multi-head Latent Attention (MLA) and DeepSeekMoE architec-
tures, which were thoroughly validated in DeepSeek-V2. Furthermore, DeepSeek-V3 pioneers
-------
training, DeepSeek-V3 adopts Multi-head Latent Attention (MLA) and DeepSeekMoE architec-
tures, which were thoroughly validated in DeepSeek-V2. Furthermore, DeepSeek-V3 pioneers
an auxiliary-loss-free strategy for load balancing and sets a multi-token prediction training
objective for stronger performance. We pre-train DeepSeek-V3 on 14.8 trillion diverse and
high-quality tokens, followed by Supervised Fine-Tuning and Reinforcement Learning stages to
-------
objective for stronger performance. We pre-train DeepSeek-V3 on 14.8 trillion diverse and
high

<div class="alert alert-info">
类似 LlamaIndex，LangChain 也提供了丰富的 <code><a href="https://python.langchain.com/v0.2/docs/how_to/#document-loaders">Document Loaders</a></code> 和 <code><a href="https://python.langchain.com/v0.2/docs/how_to/#text-splitters">Text Splitters</a></code>。
</div>


### 2.3、向量数据库与向量检索


In [ ]:
# !pip install dashscope
# !pip install faiss-cpu

In [33]:
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyMuPDFLoader

# 加载文档
loader = PyMuPDFLoader("./data/deepseek-v3-1-4.pdf")
pages = loader.load_and_split()

# 文档切分
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=200,
    length_function=len,
    add_start_index=True,
)

texts = text_splitter.create_documents(
    [page.page_content for page in pages[:1]]
)

# 灌库
embeddings = DashScopeEmbeddings(
    model="text-embedding-v1", dashscope_api_key=os.getenv("DASHSCOPE_API_KEY")
)
index = FAISS.from_documents(texts, embeddings)

# 检索 top-5 结果
retriever = index.as_retriever(search_kwargs={"k": 5})

docs = retriever.invoke("deepseek v3有多少参数")

for doc in docs:
    print(doc.page_content)
    print("----")

22.6
71.6
49.0
80.0
23.3
24.8
23.8
73.3
51.1
73.8
23.3
25.3
24.5
72.6
49.9
74.6
9.3
23.6
38.8
78.0
65.0
78.3
16.0
20.3
50.8
DeepSeek-V3
DeepSeek-V2.5
Qwen2.5-72B-Inst
Llama-3.1-405B-Inst
GPT-4o-0513
Claude-3.5-Sonnet-1022
Figure 1 | Benchmark performance of DeepSeek-V3 and its counterparts.
arXiv:2412.19437v2  [cs.CL]  18 Feb 2025
----
objective for stronger performance. We pre-train DeepSeek-V3 on 14.8 trillion diverse and
high-quality tokens, followed by Supervised Fine-Tuning and Reinforcement Learning stages to
fully harness its capabilities. Comprehensive evaluations reveal that DeepSeek-V3 outperforms
other open-source models and achieves performance comparable to leading closed-source
models. Despite its excellent performance, DeepSeek-V3 requires only 2.788M H800 GPU hours
----
training, DeepSeek-V3 adopts Multi-head Latent Attention (MLA) and DeepSeekMoE architec-
tures, which were thoroughly validated in DeepSeek-V2. Furthermore, DeepSeek-V3 pioneers
an auxiliary-loss-free st

更多的三方检索组件链接，参考：https://python.langchain.com/docs/integrations/vectorstores/


### 2.4 小结

1. 文档处理部分，建议在实际应用中详细测试后使用
2. 与向量数据库的链接部分本质是接口封装，向量数据库需要自己选型


## 3. Chain 和 LangChain Expression Language (LCEL)


LangChain Expression Language（LCEL）是一种声明式语言，可轻松组合不同的调用顺序构成 Chain。LCEL 自创立之初就被设计为能够支持将原型投入生产环境，**无需代码更改**，从最简单的“提示+LLM”链到最复杂的链（已有用户成功在生产环境中运行包含数百个步骤的 LCEL Chain）。

LCEL 的一些亮点包括：

1. **流支持**：使用 LCEL 构建 Chain 时，你可以获得最佳的首个令牌时间（即从输出开始到首批输出生成的时间）。对于某些 Chain，这意味着可以直接从 LLM 流式传输令牌到流输出解析器，从而以与 LLM 提供商输出原始令牌相同的速率获得解析后的、增量的输出。

2. **异步支持**：任何使用 LCEL 构建的链条都可以通过同步 API（例如，在 Jupyter 笔记本中进行原型设计时）和异步 API（例如，在 LangServe 服务器中）调用。这使得相同的代码可用于原型设计和生产环境，具有出色的性能，并能够在同一服务器中处理多个并发请求。

3. **优化的并行执行**：当你的 LCEL 链条有可以并行执行的步骤时（例如，从多个检索器中获取文档），我们会自动执行，无论是在同步还是异步接口中，以实现最小的延迟。

4. **重试和回退**：为 LCEL 链的任何部分配置重试和回退。这是使链在规模上更可靠的绝佳方式。目前我们正在添加重试/回退的流媒体支持，因此你可以在不增加任何延迟成本的情况下获得增加的可靠性。

5. **访问中间结果**：对于更复杂的链条，访问在最终输出产生之前的中间步骤的结果通常非常有用。这可以用于让最终用户知道正在发生一些事情，甚至仅用于调试链条。你可以流式传输中间结果，并且在每个 LangServe 服务器上都可用。

6. **输入和输出模式**：输入和输出模式为每个 LCEL 链提供了从链的结构推断出的 Pydantic 和 JSONSchema 模式。这可以用于输入和输出的验证，是 LangServe 的一个组成部分。

7. **无缝 LangSmith 跟踪集成**：随着链条变得越来越复杂，理解每一步发生了什么变得越来越重要。通过 LCEL，所有步骤都自动记录到 LangSmith，以实现最大的可观察性和可调试性。

8. **无缝 LangServe 部署集成**：任何使用 LCEL 创建的链都可以轻松地使用 LangServe 进行部署。

原文：https://python.langchain.com/docs/expression_language/


### 3.1 Pipeline 式调用 PromptTemplate, LLM 和 OutputParser

In [35]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from pydantic import BaseModel, Field
from typing import List, Dict, Optional
from enum import Enum
import json
from langchain.chat_models import init_chat_model

In [36]:
# 输出结构
class SortEnum(str, Enum):
    data = 'data'
    price = 'price'


class OrderingEnum(str, Enum):
    ascend = 'ascend'
    descend = 'descend'


class Semantics(BaseModel):
    name: Optional[str] = Field(description="流量包名称", default=None)
    price_lower: Optional[int] = Field(description="价格下限", default=None)
    price_upper: Optional[int] = Field(description="价格上限", default=None)
    data_lower: Optional[int] = Field(description="流量下限", default=None)
    data_upper: Optional[int] = Field(description="流量上限", default=None)
    sort_by: Optional[SortEnum] = Field(description="按价格或流量排序", default=None)
    ordering: Optional[OrderingEnum] = Field(
description="升序或降序排列", default=None)


# Prompt 模板
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个语义解析器。你的任务是将用户的输入解析成JSON表示。不要回答用户的问题。"),
        ("human", "{text}"),
    ]
)

# 模型
llm = init_chat_model("deepseek-chat", model_provider="deepseek")

structured_llm = llm.with_structured_output(Semantics)

# LCEL 表达式
runnable = (
    {"text": RunnablePassthrough()} | prompt | structured_llm
)

# 直接运行
ret = runnable.invoke("不超过100元的流量大的套餐有哪些")
print(
    json.dumps(
        ret.model_dump(),
        indent = 4,
        ensure_ascii=False
    )
)

{
    "name": null,
    "price_lower": null,
    "price_upper": 100,
    "data_lower": null,
    "data_upper": null,
    "sort_by": "data",
    "ordering": "descend"
}


<div class="alert alert-success">
<b>使用 LCEL 的价值，也就是 LangChain 的核心价值。</b> <br />
官方从不同角度给出了举例说明：<a href="https://python.langchain.com/docs/concepts/lcel/">https://python.langchain.com/docs/concepts/lcel/</a>
</div>


### 3.2 用 LCEL 实现 RAG


In [40]:
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
# from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.dashscope import DashScopeEmbeddings

# 加载文档
loader = PyMuPDFLoader("./data/deepseek-v3-1-4.pdf")
pages = loader.load_and_split()

# 文档切分
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=200,
    length_function=len,
    add_start_index=True,
)

texts = text_splitter.create_documents(
    [page.page_content for page in pages[:1]]
)

# 灌库
embeddings = DashScopeEmbeddings(
    model="text-embedding-v1", dashscope_api_key=os.getenv("DASHSCOPE_API_KEY")
)
db = FAISS.from_documents(texts, embeddings)

# 检索 top-5 结果
retriever = db.as_retriever(search_kwargs={"k": 5})

In [44]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Prompt模板
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Chain
rag_chain = (
    {"question": RunnablePassthrough(), "context": retriever}
    | prompt
    | llm
    | StrOutputParser()
)

response = rag_chain.stream("deepseek V3有多少参数")
for value in response:
    print(value, end='', flush=True)

DeepSeek-V3 是一个拥有 6710 亿总参数的混合专家（MoE）语言模型，每个 token 激活 370 亿参数。

### 3.3 用 LCEL 实现模型切换（工厂模式）


In [51]:
from langchain_core.runnables.utils import ConfigurableField
from langchain_community.chat_models import QianfanChatEndpoint
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage
import os

# 直接设置环境变量
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_BASE_URL"] = "https://api.apiyi.com/v1"

# 模型1
ds_model = init_chat_model("deepseek-chat", model_provider="deepseek")

# 模型2
gpt_model = init_chat_model("gpt-4o-mini", model_provider="openai")


# 通过 configurable_alternatives 按指定字段选择模型
model = gpt_model.configurable_alternatives(
    ConfigurableField(id="llm"), 
    default_key="gpt", 
    deepseek=ds_model,
    # claude=claude_model,
)

# Prompt 模板
prompt = ChatPromptTemplate.from_messages(
    [
        HumanMessagePromptTemplate.from_template("{query}"),
    ]
)

# LCEL
chain = (
    {"query": RunnablePassthrough()} 
    | prompt
    | model 
    | StrOutputParser()
)

# 运行时指定模型 "gpt" or "deepseek"
ret = chain.with_config(configurable={"llm": "deepseek"}).invoke("请自我介绍")

print(ret)

你好！我是DeepSeek，由深度求索公司创造的AI助手！😊

让我简单介绍一下自己：

**基本信息：**
- 我是一个纯文本AI模型，擅长理解和生成自然语言
- 知识截止到2024年7月，拥有128K的上下文处理能力

**我的能力：**
- 回答各种问题，进行深度对话
- 支持文件上传（图像、txt、pdf、ppt、word、excel等），能读取其中的文字信息进行处理
- 支持联网搜索（需要你手动开启搜索功能）
- 完全免费使用，没有收费计划

**我的特点：**
- 热情细腻的回复风格，喜欢用心帮助每一位用户
- 可以通过官方应用商店下载App使用
- 乐于助人，无论是学习、工作还是生活中的问题，都可以找我聊聊

**需要注意的：**
- 我是纯文本模型，不支持多模态识别和语音功能
- 但我会尽力通过文字为你提供最好的帮助！

有什么想了解的或者需要帮助的吗？我很乐意为你服务！✨


扩展阅读：什么是[**工厂模式**](https://www.runoob.com/design-pattern/factory-pattern.html)；[**设计模式**](https://www.runoob.com/design-pattern/design-pattern-intro.html)概览。


<div class="alert alert-warning">
<b>思考：</b>从模块间解依赖角度，LCEL的意义是什么？
</div>


### 3.4 通过 LCEL，还可以实现

1. 配置运行时变量：https://python.langchain.com/docs/how_to/configure/
2. 故障回退：https://python.langchain.com/docs/how_to/fallbacks/
3. 并行调用：https://python.langchain.com/docs/how_to/parallel/
4. 逻辑分支：https://python.langchain.com/docs/how_to/routing/
5. 动态创建 Chain: https://python.langchain.com/docs/how_to/dynamic_chain/

更多例子：https://python.langchain.com/docs/how_to/lcel_cheatsheet/

## 4. LangChain 与 LlamaIndex 的错位竞争

- LangChain 侧重与 LLM 本身交互的封装
  - Prompt、LLM、Message、OutputParser 等工具丰富
  - 在数据处理和 RAG 方面提供的工具相对粗糙
  - 主打 LCEL 流程封装
  - 配套 Agent、LangGraph 等智能体与工作流工具
  - 另有 LangServe 部署工具和 LangSmith 监控调试工具
- LlamaIndex 侧重与数据交互的封装
  - 数据加载、切割、索引、检索、排序等相关工具丰富
  - Prompt、LLM 等底层封装相对单薄
  - 配套实现 RAG 相关工具
  - 同样配套智能体与工作流工具
  - 提供 LlamaDeploy 部署工具，通过与三方合作提供过程监控调试工具

## 5. 总结

1. LangChain 随着版本迭代可用性有明显提升
2. 使用 LangChain 要注意维护自己的 Prompt，尽量 Prompt 与代码逻辑解依赖
3. 它的内置基础工具，建议充分测试效果后再决定是否使用
